In [1]:
using CABLAB
ds=Cube("/home/jovyan/work/datacube/cablab-datacube-0.2.3/high-res/")
d2=getCubeData(ds,variable=["Biosphere"],region="Poland")
d=d2

Data Cube view with the following dimensions
Lon                 Axis with 121 Elements from 14.082557000000008 to 24.08251700000001
Lat                 Axis with 70 Elements from 54.833473999999995 to 49.083496999999994
Time                Axis with 506 Elements from 2001-01-01T00:00:00 to 2011-12-27T00:00:00
Variable            Axis with elements: bare_soil_evaporation black_sky_albedo burnt_area c_emissions evaporation evaporative_stress gross_primary_productivity interception_loss land_surface_temperature latent_energy net_ecosystem_exchange open_water_evaporation root_moisture sensible_heat soil_moisture surface_moisture terrestrial_ecosystem_respiration transpiration white_sky_albedo 
Total size: 388.29 MB


In [2]:
include("loadLC.jl")

In-Memory data cube with the following dimensions
Lon                 Axis with 121 Elements from 14.082557000000008 to 24.08251700000001
Lat                 Axis with 70 Elements from 54.833473999999995 to 49.083496999999994
Total size: 16.54 KB


In [3]:
@loadOrGenerate v1=>"variance_pca" r1=>"rotation_pca" proj1=>"Projected" begin
  p1=cubePCA(d,noutdim=3)
  v1=explained_variance(p1)
  r1=rotation_matrix(p1)
    proj1=transformPCA(p1,d)  
end

true

In [4]:
plotXY(v1,xaxis="pc")

In [5]:
plotXY(r1,xaxis="pc",group="variable")

In [6]:
proj1

Temporary Data Cube with the following dimensions
PC                  Axis with elements: PC 1 PC 2 PC 3 
Lon                 Axis with 121 Elements from 14.082557000000008 to 24.08251700000001
Time                Axis with 506 Elements from 2001-01-01T00:00:00 to 2011-12-27T00:00:00
Lat                 Axis with 70 Elements from 54.833473999999995 to 49.083496999999994
Total size: 61.31 MB


In [9]:
using OnlineStats
index = mapCube(Mean,proj1,by=(TimeAxis,CategoricalAxis{String,:PC}))

In-Memory data cube with the following dimensions
PC                  Axis with elements: PC 1 PC 2 PC 3 
Time                Axis with 506 Elements from 2001-01-01T00:00:00 to 2011-12-27T00:00:00
Total size: 13.34 KB


In [10]:
plotTS(index)

Interact.Options{:Dropdown,Int64}(Signal{Int64}(0, nactions=1),"Group",0,"None",Interact.OptionDict(DataStructures.OrderedDict("None"=>0,"PC"=>1),Dict(0=>"None",1=>"PC")),Any[],Any[],true,"horizontal")

Interact.Options{:Dropdown,Int64}(Signal{Int64}(2, nactions=1),"PC",2,"PC 2",Interact.OptionDict(DataStructures.OrderedDict("PC 2"=>2,"PC 1"=>1,"PC 3"=>3),Dict(2=>"PC 2",3=>"PC 3",1=>"PC 1")),Any[],Any[],true,"horizontal")

In [11]:
@loadOrGenerate vlc=>"variance_by_landCover" rlc=>"rotation_by_landcover" begin
  plandclass=cubePCA(d,noutdim=3,by=[lc])
  vlc=explained_variance(plandclass)
  rlc=rotation_matrix(plandclass)
end

Temporary Data Cube with the following dimensions
Variable            Axis with elements: bare_soil_evaporation black_sky_albedo burnt_area c_emissions evaporation evaporative_stress gross_primary_productivity interception_loss land_surface_temperature latent_energy net_ecosystem_exchange open_water_evaporation root_moisture sensible_heat soil_moisture surface_moisture terrestrial_ecosystem_respiration transpiration white_sky_albedo 
PC                  Axis with elements: PC 1 PC 2 PC 3 
Landcover           Axis with elements: Tree cover, needleleaved, evergreen Tree cover, mixed leaf Bare areas Grassland Cropland, Herbacous, Tree/shrub Urban areas Water bodies Tree cover, broadleaved, deciduous Shrub or herbaceous cover, flooded 
Total size: 2.5 KB


In [12]:
plotXY(vlc,xaxis="pc",group="landcover")

In [13]:
plotXY(rlc,xaxis="pc",group="variable")

Interact.Options{:Dropdown,Int64}(Signal{Int64}(3, nactions=1),"Landcover",3,"Bare areas",Interact.OptionDict(DataStructures.OrderedDict("Bare areas"=>3,"Tree cover, mixed leaf"=>2,"Tree cover, needleleaved, evergreen"=>1,"Cropland, Herbacous, Tree/shrub"=>5,"Water bodies"=>7,"Grassland"=>4,"Shrub or herbaceous cover, flooded"=>9,"Tree cover, broadleaved, deciduous"=>8,"Urban areas"=>6),Dict(7=>"Water bodies",4=>"Grassland",9=>"Shrub or herbaceous cover, flooded",2=>"Tree cover, mixed leaf",3=>"Bare areas",5=>"Cropland, Herbacous, Tree/shrub",8=>"Tree cover, broadleaved, deciduous",6=>"Urban areas",1=>"Tree cover, needleleaved, evergreen")),Any[],Any[],true,"horizontal")

Masks don't have to be spatial

In [14]:
tax = d.timeAxis
function getSeason(x::Dates.TimeType)
  m = Dates.month(x)
  in(m,(12,1,2)) ? 1 : in(m,(3,4,5)) ? 2 : in(m,(6,7,8)) ? 3 : 4
end
seasonmask=CubeMem(CubeAxis[tax],getSeason.(tax.values),zeros(UInt8,length(tax)))
seasonmask.properties["labels"]=Dict(1=>"Spring",2=>"Summer",3=>"Autumn",4=>"Winter")
seasonmask.properties["name"]="Season"
plotTS(seasonmask)

In [15]:
@loadOrGenerate vseason=>"variance_by_season" rseason=>"rotation_by_season" projseason=>"Projected_by_season" begin
  pseason=cubePCA(d,noutdim=3,by=[seasonmask])
  vseason=explained_variance(pseason)
  rseason=rotation_matrix(pseason)
  projseason=transformPCA(pseason,d)
end

Temporary Data Cube with the following dimensions
PC                  Axis with elements: PC 1 PC 2 PC 3 
Time                Axis with 506 Elements from 2001-01-01T00:00:00 to 2011-12-27T00:00:00
Lon                 Axis with 121 Elements from 14.082557000000008 to 24.08251700000001
Lat                 Axis with 70 Elements from 54.833473999999995 to 49.083496999999994
Total size: 61.31 MB


In [16]:
plotXY(vseason,xaxis="PC",group="season")

In [17]:
@loadOrGenerate vtime=>"timePCA_variance" rtime=>"timePCA_rotation" projtime=>"timePCA_Projected" begin
  ptime = cubePCA(d,noutdims=5,by=[VariableAxis],MDAxis=TimeAxis)
  vtime  = explained_variance(ptime)
  rtime  = rotation_matrix(ptime)
  projtime = transformPCA(ptime,d)
end

Temporary Data Cube with the following dimensions
PC                  Axis with elements: PC 1 PC 2 PC 3 PC 4 PC 5 
Lon                 Axis with 121 Elements from 14.082557000000008 to 24.08251700000001
Lat                 Axis with 70 Elements from 54.833473999999995 to 49.083496999999994
Variable            Axis with elements: bare_soil_evaporation black_sky_albedo burnt_area c_emissions evaporation evaporative_stress gross_primary_productivity interception_loss land_surface_temperature latent_energy net_ecosystem_exchange open_water_evaporation root_moisture sensible_heat soil_moisture surface_moisture terrestrial_ecosystem_respiration transpiration white_sky_albedo 
Total size: 3.84 MB


In [18]:
plotXY(vtime,xaxis="pc",group="variable")

In [20]:
plotTS(rtime,pc=1,group="var")